Implement various metrics to evaluate the performance of the RL model & H-network on how good they can mask user's load

Tentative metrics

Privacy metrics
- Mutual Information (either follows paper's evaluation, or other methods?)
- Matthews correlation coefficient (MCC) ? But we need to train an attacker as well. (Ruichang used this in a previous paper, PLS-DDPG)
- KL-divergence? Should be easy to compute

Monetary Cost metrics
- Compute the extra cost incured for the battery operation (i.e. compute the masked load vs original grid load)

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime
import json
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression
import matplotlib
matplotlib.use('Agg')       # Use non-interactive backend for matplotlib to save memory

from utils import print_log

In [2]:
%load_ext autoreload
%autoreload 2

---

Load the train/valid/test per-episode log

In [3]:
expt_datetime = datetime(2025, 7, 23, 18, 33, 9)
result_phrase = "train"  # or "test" or "validate"

expt_folder = Path("experiments") / expt_datetime.strftime("%Y%m%d_%H%M%S") / ("logs_" + result_phrase)

if not expt_folder.exists():
    print_log(f"Experiment folder {expt_folder} does not exist. Please check the path.", "error")

In [4]:
env_config_path = expt_folder / "env_config.json"
if not env_config_path.exists():
    print_log(f"Environment config file {env_config_path} does not exist. Please check the path.", "error")

# Load environment configuration
with open(env_config_path, 'r') as f:
    env_config = json.load(f)

# Display environment configuration
print_log("Environment Configuration:")
for key, value in env_config.items():
    print_log(f"{key}: {value}")

[2025-07-23 13:49:37:321] Environment Configuration:
[2025-07-23 13:49:37:321] battery: {'capacity': 8.0, 'max_charging_rate': 4.0, 'max_discharging_rate': 4.0, 'efficiency': 1.0, 'initial_soc': 1.0}
[2025-07-23 13:49:37:321] reward_lambda: 0.5
[2025-07-23 13:49:37:321] h_network_type: HNetworkType.H_NETWORK2
[2025-07-23 13:49:37:321] init_soc: 0.15


In [4]:
# max_battery_capacity = env_config.get("battery").get("capacity", 8)
max_battery_capacity = 8

In [ ]:
episode_info_folder = expt_folder / "episode_info"
if episode_info_folder.exists():
    episode_info_files = sorted(list(episode_info_folder.glob("*.json")))
    episode_info_dfs = []

    for file in episode_info_files:
        with open(file, "r") as f:
            episode_info = json.load(f)
            df = pd.DataFrame(episode_info)
            # add a column for the episode number
            df['episode'] = int(file.stem.split('_')[1])  # Assuming the file name is like "episode_0_info.json", "episode_1_info.json", etc.
            # convert the datetime strings to datetime objects using python isoformat
            df['datetime'] = pd.to_datetime(df['datetime'], format='ISO8601')

            # fix to align with episode_df.pkl
            # we shift "grid_load (W)", "action (kW)", "battery_action (kW)", "reward", "f_signal", "g_signal" columns to the a timestep forward, then drop the last row for each episode
            df['grid_load (W)'] = df['grid_load (W)'].shift(-1)
            df['action (kW)'] = df['action (kW)'].shift(-1)
            df['battery_action (kW)'] = df['battery_action (kW)'].shift(-1)
            df['reward'] = df['reward'].shift(-1)
            df['f_signal'] = df['f_signal'].shift(-1)
            df['g_signal'] = df['g_signal'].shift(-1)
            # remove the last row for each episode
            df = df[:-1]

            episode_info_dfs.append(df)

            print_log(f"Loaded episode info from {file.name}")

    # Concatenate all DataFrames into one
    episode_info_df = pd.concat(episode_info_dfs, ignore_index=True)
    

[2025-07-24 17:25:12:909] Loaded episode info from episode_0001_info.json
[2025-07-24 17:25:12:917] Loaded episode info from episode_0002_info.json
[2025-07-24 17:25:12:924] Loaded episode info from episode_0003_info.json
[2025-07-24 17:25:12:931] Loaded episode info from episode_0004_info.json
[2025-07-24 17:25:12:937] Loaded episode info from episode_0005_info.json
[2025-07-24 17:25:12:944] Loaded episode info from episode_0006_info.json
[2025-07-24 17:25:12:950] Loaded episode info from episode_0007_info.json
[2025-07-24 17:25:12:956] Loaded episode info from episode_0008_info.json
[2025-07-24 17:25:12:962] Loaded episode info from episode_0009_info.json
[2025-07-24 17:25:12:968] Loaded episode info from episode_0010_info.json
[2025-07-24 17:25:12:975] Loaded episode info from episode_0011_info.json
[2025-07-24 17:25:12:981] Loaded episode info from episode_0012_info.json
[2025-07-24 17:25:12:988] Loaded episode info from episode_0013_info.json
[2025-07-24 17:25:12:994] Loaded episo

In [13]:
episode_info_dfs[0]

,episode_index,current_step,datetime,battery_soc (%),battery_soc (kWh),user_load (W),grid_load (W),action (kW),battery_action (kW),reward,f_signal,g_signal,f_signal-predicted_mean,f_signal-predicted_log_var,f_signal-target,TimeLimit.truncated,episode
0,96,0,2013-08-28 00:00:02,0.150000,1.200000,304.000000,804.377238,0.500377,0.500377,-0.000239,-0.000364,0.000842,NaN,NaN,NaN,NaN,1
1,96,1,2013-08-28 00:01:02,0.151042,1.208340,301.620664,2423.906984,2.122286,2.122286,-0.001717,-0.000138,0.003573,-0.023197,0.0,-0.050192,False,1
2,96,2,2013-08-28 00:02:02,0.155464,1.243711,302.718374,3753.618213,3.450900,3.450900,-0.002885,-0.000039,0.005809,-0.030296,0.0,-0.046925,False,1
3,96,3,2013-08-28 00:03:02,0.162653,1.301226,307.954288,4307.954288,4.000000,4.000000,-0.003357,-0.000019,0.006733,-0.040169,0.0,-0.031338,False,1
4,96,4,2013-08-28 00:04:02,0.170987,1.367893,300.548825,3280.452285,2.979903,2.979903,-0.002507,-0.000002,0.005016,-0.047256,0.0,-0.053383,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,96,1434,2013-08-28 23:54:02,1.000000,8.000000,212.838056,0.000000,-2.247849,-0.212838,0.022059,-0.044476,0.000358,-0.009926,0.0,-0.314484,False,1
1435,96,1435,2013-08-28 23:55:02,0.999557,7.996453,212.814858,425.652914,0.349726,0.212838,0.022147,-0.044653,0.000358,-0.009276,0.0,-0.314553,False,1
1436,96,1436,2013-08-28 23:56:02,1.000000,8.000000,212.412497,212.412497,1.007984,0.000000,0.037799,-0.075598,0.000000,-0.009837,0.0,-0.315751,False,1
1437,96,1437,2013-08-28 23:57:02,1.000000,8.000000,179.139330,0.000000,-1.540693,-0.179139,0.050390,-0.101081,0.000302,-0.009674,0.0,-0.414800,False,1


Run the following cells

Mutual Information implementation

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html

In the paper `Privacy-Cost Management in Smart Meters With Mutual-Information-Based Reinforcement Learning`, the parameter $k$ (in KNN estimation) is set to 4

X will be the masked grid load; y will be the user load

In [6]:
def calculate_mutual_information_per_episode(episode_info_dfs, k=4):
    """
    Calculate mutual information between masked grid load and user load for each episode.
    
    Args:
        episode_info_dfs: List of episode DataFrames
        k: Number of neighbors for KNN estimation (default=4 as per paper)
    
    Returns:
        Dictionary containing:
        - 'mi_values': List of mutual information values for each episode
        - 'episode_indices': List of episode indices from the dataset
        - 'episode_info_indices': List of indices in the episode_info_dfs list
    """
    mi_results = {
        'mi_values': [],
        'episode_indices': [],
        'episode_info_indices': []
    }
    
    for i, episode_df in enumerate(episode_info_dfs):
        try:
            # Get the episode index from the dataframe
            episode_idx = episode_df['episode_index'].iloc[0] if 'episode_index' in episode_df.columns else episode_df['episode'].iloc[0]
            
            # Create a copy to avoid modifying the original dataframe
            df_clean = episode_df.copy()
            
            # Remove rows with NaN values in either grid_load or user_load columns
            df_clean = df_clean.dropna(subset=['grid_load (W)', 'user_load (W)'])
            
            # Check if we have enough data points after removing NaN values
            if len(df_clean) < k + 1:  # Need at least k+1 points for KNN
                print_log(f"Episode {i} (episode_idx: {episode_idx}): Not enough valid data points ({len(df_clean)}) after removing NaN values")
                mi_results['mi_values'].append(np.nan)
                mi_results['episode_indices'].append(episode_idx)
                mi_results['episode_info_indices'].append(i)
                continue
            
            # Extract the features: masked grid load (X) and user load (y)
            X = df_clean['grid_load (W)'].values.reshape(-1, 1)  # Masked grid load (features)
            y = df_clean['user_load (W)'].values  # User load (continuous target)
            
            # Calculate mutual information using scikit-learn for regression
            # mutual_info_regression handles continuous variables directly
            mi = mutual_info_regression(X, y, n_neighbors=k, random_state=42)
            
            mi_results['mi_values'].append(mi[0])  # mi is an array, we take the first element
            mi_results['episode_indices'].append(episode_idx)
            mi_results['episode_info_indices'].append(i)
            
            if i % 100 == 0:  # Print progress every 100 episodes
                print_log(f"Processed episode {i}/{len(episode_info_dfs)} (episode_idx: {episode_idx})")

            del df_clean  # Free memory after processing
                
        except Exception as e:
            # Get episode index even in case of error
            episode_idx = episode_df['episode_index'].iloc[0] if 'episode_index' in episode_df.columns else episode_df['episode'].iloc[0] if 'episode' in episode_df.columns else i
            print_log(f"Error calculating MI for episode {i} (episode_idx: {episode_idx}): {str(e)}")
            mi_results['mi_values'].append(np.nan)  # Add NaN for failed calculations
            mi_results['episode_indices'].append(episode_idx)
            mi_results['episode_info_indices'].append(i)
    
    print_log(f"Calculated mutual information for {len(mi_results['mi_values'])} episodes")
    return mi_results

In [7]:
# Calculate mutual information for all episodes
print_log("Starting mutual information calculation...")
mi_results = calculate_mutual_information_per_episode(episode_info_dfs, k=4)

# Extract the components from the results
mutual_info_values = mi_results['mi_values']
episode_indices = mi_results['episode_indices']
episode_info_indices = mi_results['episode_info_indices']

# Create a comprehensive DataFrame with episode, episode_index, and MI values
mi_df = pd.DataFrame({
    'episode_info_index': episode_info_indices,         # Index in the episode_info_dfs list
    'episode_index': episode_indices,                   # Episode index from the dataset
    'mutual_information': mutual_info_values            # MI values
})

print_log(f"Mutual Information Statistics:")
print_log(f"Mean MI: {np.nanmean(mutual_info_values):.6f}")
print_log(f"Std MI: {np.nanstd(mutual_info_values):.6f}")
print_log(f"Min MI: {np.nanmin(mutual_info_values):.6f}")
print_log(f"Max MI: {np.nanmax(mutual_info_values):.6f}")

print_log(f"DataFrame shape: {mi_df.shape}")
print_log(f"Episode index range: {mi_df['episode_index'].min()} to {mi_df['episode_index'].max()}")

mi_df.head()

[2025-07-24 17:25:17:469] Starting mutual information calculation...
[2025-07-24 17:25:17:476] Processed episode 0/605 (episode_idx: 96)
[2025-07-24 17:25:17:897] Processed episode 100/605 (episode_idx: 42)
[2025-07-24 17:25:18:323] Processed episode 200/605 (episode_idx: 75)
[2025-07-24 17:25:18:726] Processed episode 300/605 (episode_idx: 42)
[2025-07-24 17:25:19:120] Processed episode 400/605 (episode_idx: 58)
[2025-07-24 17:25:19:506] Processed episode 500/605 (episode_idx: 150)
[2025-07-24 17:25:19:886] Processed episode 600/605 (episode_idx: 80)
[2025-07-24 17:25:19:901] Calculated mutual information for 605 episodes
[2025-07-24 17:25:19:902] Mutual Information Statistics:
[2025-07-24 17:25:19:902] Mean MI: 1.461475
[2025-07-24 17:25:19:902] Std MI: 0.802302
[2025-07-24 17:25:19:902] Min MI: 0.087612
[2025-07-24 17:25:19:902] Max MI: 3.209402
[2025-07-24 17:25:19:902] DataFrame shape: (605, 3)
[2025-07-24 17:25:19:902] Episode index range: 0 to 161


,episode_info_index,episode_index,mutual_information
0,0,96,0.814674
1,1,8,0.639439
2,2,147,0.728555
3,3,35,0.749873
4,4,41,0.413370


Apart from the Mutual Information, we also want to compute the extra cost incurred from our policy

The cost can be computed as

$$\int (B_t - Y_t)h_t \triangle_t$$

where $B_t$ is the battery state of charge at time t; $Y_t$ is the user load at time t, $h_t$ is the time-of-use price per kWh, $\triangle_t$ is the time passed 

To compute the electricity cost, we copy the electricity cost function from our environment and slightly modify it for our purpose

In [8]:
from datetime import time, datetime

def calculate_extra_cost_per_episode(episode_info_dfs):
    """
    Calculate extra electricity cost incurred by battery actions for each episode.
    
    The extra cost is computed using the _g_signal approach where:
    - We calculate the cost difference between masked grid load and user load
    - The difference (grid_load - user_load) represents the battery charging/discharging effect
    - Cost is computed using time-of-use pricing with proper time weighting
    
    Args:
        episode_info_dfs: List of episode DataFrames
    
    Returns:
        Dictionary containing:
        - 'cost_values': List of extra cost values for each episode (in currency units)
        - 'episode_indices': List of episode indices from the dataset
        - 'episode_info_indices': List of indices in the episode_info_dfs list
    """
    
    # Helper function to get weighted electricity cost (adapted from environment)
    def get_weighted_electricity_cost(s_t_datetime:datetime, s_t_plus_1_datetime:datetime) -> float:
        """
        Calculate the weighted electricity cost based on time-of-use pricing.
        Returns: delta_t * avg_price_per_kWh for the time period
        """
        
        time_of_use_prices = {
            (time(0,0,0,0), time(7,0,0,0)): 0.101,  # off-peak price
            (time(7,0,0,0), time(11,0,0,0)): 0.208,   # peak price
            (time(11,0,0,0), time(17,0,0,0)): 0.144,  # mid-peak price
            (time(17,0,0,0), time(19,0,0,0)): 0.208,   # peak price
            (time(19,0,0,0), time(23,59,59,999999)): 0.101,  # evening price
        }

        total_cost = 0.0
        for start_time, end_time in time_of_use_prices.keys():
            if s_t_datetime.time() <= end_time and s_t_plus_1_datetime.time() >= start_time:
                # Calculate the overlap between the time intervals
                overlap_start = max(s_t_datetime, datetime.combine(s_t_datetime.date(), start_time))
                overlap_end = min(s_t_plus_1_datetime, datetime.combine(s_t_plus_1_datetime.date(), end_time))
                
                if overlap_start < overlap_end:
                    overlap_duration = (overlap_end - overlap_start).total_seconds() / 3600.0  # Convert to hours
                    price_per_kwh = time_of_use_prices[(start_time, end_time)]
                    total_cost += overlap_duration * price_per_kwh

        return total_cost
    
    # Improved _g_signal function for cost calculation
    def g_signal(s_t_datetime:datetime, s_t_plus_1_datetime:datetime, power_kw:float) -> float:
        """
        Calculate the cost signal based on elapsed time and power used.
        """
        if s_t_datetime >= s_t_plus_1_datetime:
            return 0.0  # No time elapsed, no cost
        
        weighted_cost = get_weighted_electricity_cost(s_t_datetime, s_t_plus_1_datetime)
        return weighted_cost * power_kw
    
    cost_results = {
        'cost_values': [],
        'episode_indices': [],
        'episode_info_indices': []
    }
    
    for i, episode_df in enumerate(episode_info_dfs):
        try:
            # Get the episode index from the dataframe
            episode_idx = episode_df['episode_index'].iloc[0] if 'episode_index' in episode_df.columns else episode_df['episode'].iloc[0]
            
            # Create a copy to avoid modifying the original dataframe
            df_clean = episode_df.copy()
            
            # Remove rows with NaN values in required columns
            required_columns = ['grid_load (W)', 'user_load (W)', 'datetime']
            df_clean = df_clean.dropna(subset=required_columns)
            
            # Check if we have enough data points
            if len(df_clean) < 2:  # Need at least 2 points to calculate time differences
                print_log(f"Episode {i} (episode_idx: {episode_idx}): Not enough valid data points ({len(df_clean)}) after removing NaN values")
                cost_results['cost_values'].append(np.nan)
                cost_results['episode_indices'].append(episode_idx)
                cost_results['episode_info_indices'].append(i)
                continue
            
            # Sort by datetime to ensure proper time ordering
            df_clean = df_clean.sort_values('datetime').reset_index(drop=True)
            
            total_extra_cost = 0.0
            total_grid_cost = 0.0  # Cost of actual grid consumption
            total_user_cost = 0.0  # Cost if no battery was used
            
            # Calculate cost for each time step using _g_signal approach
            for j in range(len(df_clean) - 1):
                # Current and next timestamps
                current_time = df_clean.iloc[j]['datetime']
                next_time = df_clean.iloc[j + 1]['datetime']
                
                # Power values (convert from W to kW)
                grid_load_kw = df_clean.iloc[j]['grid_load (W)'] / 1000.0
                user_load_kw = df_clean.iloc[j]['user_load (W)'] / 1000.0
                
                # Calculate costs using g_signal approach
                grid_cost = g_signal(current_time, next_time, grid_load_kw)
                user_cost = g_signal(current_time, next_time, user_load_kw)
                
                # Accumulate costs
                total_grid_cost += grid_cost
                total_user_cost += user_cost
            
            # Extra cost is the difference between what we actually paid vs what we would have paid
            total_extra_cost = abs(total_grid_cost - total_user_cost)
            
            cost_results['cost_values'].append(total_extra_cost)
            cost_results['episode_indices'].append(episode_idx)
            cost_results['episode_info_indices'].append(i)
            
            if i % 100 == 0:  # Print progress every 100 episodes
                print_log(f"Processed episode {i}/{len(episode_info_dfs)} (episode_idx: {episode_idx}), Extra cost: ${total_extra_cost:.4f}")

            del df_clean  # Free memory after processing
                
        except Exception as e:
            # Get episode index even in case of error
            episode_idx = episode_df['episode_index'].iloc[0] if 'episode_index' in episode_df.columns else episode_df['episode'].iloc[0] if 'episode' in episode_df.columns else i
            print_log(f"Error calculating extra cost for episode {i} (episode_idx: {episode_idx}): {str(e)}")
            cost_results['cost_values'].append(np.nan)  # Add NaN for failed calculations
            cost_results['episode_indices'].append(episode_idx)
            cost_results['episode_info_indices'].append(i)
    
    print_log(f"Calculated extra electricity cost for {len(cost_results['cost_values'])} episodes")
    return cost_results

# Calculate extra cost for all episodes
print_log("Starting extra electricity cost calculation...")
cost_results = calculate_extra_cost_per_episode(episode_info_dfs)

# Extract the components from the results
extra_cost_values = cost_results['cost_values']
cost_episode_indices = cost_results['episode_indices']
cost_episode_info_indices = cost_results['episode_info_indices']

# Create a comprehensive DataFrame with episode, episode_index, and cost values
cost_df = pd.DataFrame({
    'episode_info_index': cost_episode_info_indices,         # Index in the episode_info_dfs list
    'episode_index': cost_episode_indices,                   # Episode index from the dataset
    'extra_cost': extra_cost_values                          # Extra cost values
})

print_log(f"Extra Electricity Cost Statistics:")
print_log(f"Mean Extra Cost: ${np.nanmean(extra_cost_values):.4f}")
print_log(f"Std Extra Cost: ${np.nanstd(extra_cost_values):.4f}")
print_log(f"Min Extra Cost: ${np.nanmin(extra_cost_values):.4f}")
print_log(f"Max Extra Cost: ${np.nanmax(extra_cost_values):.4f}")

print_log(f"DataFrame shape: {cost_df.shape}")
print_log(f"Episode index range: {cost_df['episode_index'].min()} to {cost_df['episode_index'].max()}")

cost_df.head()

[2025-07-24 17:25:19:975] Starting extra electricity cost calculation...
[2025-07-24 17:25:20:153] Processed episode 0/605 (episode_idx: 96), Extra cost: $0.6867
[2025-07-24 17:25:37:642] Processed episode 100/605 (episode_idx: 42), Extra cost: $0.8071
[2025-07-24 17:25:54:246] Processed episode 200/605 (episode_idx: 75), Extra cost: $0.9747
[2025-07-24 17:26:11:628] Processed episode 300/605 (episode_idx: 42), Extra cost: $0.9132
[2025-07-24 17:26:27:937] Processed episode 400/605 (episode_idx: 58), Extra cost: $0.6505
[2025-07-24 17:26:45:154] Processed episode 500/605 (episode_idx: 150), Extra cost: $0.4836
[2025-07-24 17:27:01:638] Processed episode 600/605 (episode_idx: 80), Extra cost: $0.6442
[2025-07-24 17:27:02:191] Calculated extra electricity cost for 605 episodes
[2025-07-24 17:27:02:192] Extra Electricity Cost Statistics:
[2025-07-24 17:27:02:192] Mean Extra Cost: $0.6420
[2025-07-24 17:27:02:192] Std Extra Cost: $0.1408
[2025-07-24 17:27:02:192] Min Extra Cost: $0.1593
[2

,episode_info_index,episode_index,extra_cost
0,0,96,0.686674
1,1,8,0.683787
2,2,147,0.686962
3,3,35,0.702544
4,4,41,0.686784


---

In [9]:
def plot_episode_with_psd(episode_df, episode_idx, mi_value, extra_cost_value, save_path=None, use_datetime=False, show=True):
    """
    Plot episode details with user load, grid load, and Power Spectral Density of masked grid load
    
    Args:
        episode_df: DataFrame containing episode information
        episode_idx: Episode index for title
        mi_value: Mutual information value for this episode (displayed as text)
        extra_cost_value: Extra electricity cost value for this episode (displayed as text)
        save_path: Optional path to save the figure
        use_datetime: If True, use datetime as x-axis; if False, use step numbers
        show: If True, display the plot; if False, suppress display
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 10), dpi=150)
    
    # Determine x-axis values for the top plot
    if use_datetime and 'datetime' in episode_df.columns:
        x_values = episode_df['datetime']
        x_label = 'Time'
    else:
        x_values = range(len(episode_df))
        x_label = 'Time Steps'

    episode_ittr = episode_df['episode'].iloc[0]  # Get the episode iteration number for title
    
    # First subplot: User load and Grid load (same as in expt_results.ipynb)
    ax1.plot(x_values, episode_df['user_load (W)'], label='User Load', color='blue', linewidth=2)
    ax1.plot(x_values, episode_df['grid_load (W)'], label='Grid Load (Masked)', color='pink', linewidth=2, alpha=0.8)

    ax1.set_title(f'Episode {episode_idx} @ ittr {episode_ittr}: Load Profiles Over Time (MI = {mi_value:.6f})')
    ax1.set_ylabel('Power (W)')
    ax1.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Second subplot: Power Spectral Density of masked grid load using Welch's method
    # Handle the index shift - masked load appears at iloc[1:]
    # Extract the masked grid load data, removing NaN values
    masked_load_data = episode_df['grid_load (W)'].dropna().values
    
    if len(masked_load_data) > 10:  # Need sufficient data for PSD
        # Calculate sampling frequency (assuming regular intervals)
        # If we have datetime, calculate from actual time differences
        if use_datetime and 'datetime' in episode_df.columns:
            # Calculate average time difference in seconds
            time_diffs = episode_df['datetime'].diff().dt.total_seconds().dropna()
            if len(time_diffs) > 0:
                avg_dt = time_diffs.mean()
                fs = 1.0 / avg_dt  # Sampling frequency in Hz
            else:
                fs = 1.0 / 6.0 # Default fallback (1 sample per 6 seconds)
        else:
            fs = 1.0 / 6.0  # Default sampling frequency (1 sample per 6 seconds)

        # Plot PSD using Welch's method
        # Use reasonable parameters for NFFT and overlap
        nfft = min(256, len(masked_load_data) // 4)  # NFFT size
        noverlap = nfft // 2  # 50% overlap
        
        try:
            # Plot PSD using Welch's method with normalized frequency
            # Use Fs=2.0 to get normalized frequency output (0 to 1 range)
            ax2.psd(masked_load_data, NFFT=nfft, Fs=2.0, noverlap=noverlap, 
                   window=mlab.window_hanning, scale_by_freq=True)
            ax2.set_title(f'Episode {episode_idx} @ ittr {episode_ittr}: Power Spectral Density of Masked Grid Load (Welch\'s Method)')
            ax2.set_ylabel('Power Spectral Density (dB/Hz)')
            ax2.set_xlabel('Normalized Frequency')
            ax2.grid(True, alpha=0.3)
            
            # Add MI value as text box on the right-hand side (similar position to legend)
            mi_text = f'Mutual Information (MI): {mi_value:.6f}'
            ax2.text(0.98, 0.98, mi_text, transform=ax2.transAxes, fontsize=12,
                    verticalalignment='top', horizontalalignment='right',
                    bbox=dict(boxstyle='round', facecolor='none', edgecolor='black', alpha=0.8))
            
            # Add extra cost value as text box below MI text box
            cost_text = f'Extra Cost: ${extra_cost_value:.4f}'
            ax2.text(0.98, 0.88, cost_text, transform=ax2.transAxes, fontsize=12,
                    verticalalignment='top', horizontalalignment='right',
                    bbox=dict(boxstyle='round', facecolor='none', edgecolor='black', alpha=0.8))
                    
        except Exception as e:
            # Fallback: if PSD fails, plot a simple message
            fallback_text = f'PSD calculation failed:\n{str(e)}\n\nMutual Information (MI): {mi_value:.6f}\nExtra Cost: ${extra_cost_value:.4f}'
            ax2.text(0.5, 0.5, fallback_text, 
                    transform=ax2.transAxes, fontsize=12, ha='center', va='center',
                    bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.8))
            ax2.set_title(f'Episode {episode_idx} @ ittr {episode_ittr}: PSD Analysis Failed')
            ax2.set_ylabel('N/A')
            ax2.set_xlabel('N/A')
    else:
        # Not enough data for PSD
        insufficient_text = f'Insufficient data for PSD analysis\n(only {len(masked_load_data)} valid points)\n\nMutual Information (MI): {mi_value:.6f}\nExtra Cost: ${extra_cost_value:.4f}'
        ax2.text(0.5, 0.5, insufficient_text, 
                transform=ax2.transAxes, fontsize=12, ha='center', va='center',
                bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
        ax2.set_title(f'Episode {episode_idx} @ ittr {episode_ittr}: Insufficient Data for PSD')
        ax2.set_ylabel('N/A')
        ax2.set_xlabel('N/A')
    
    # Format datetime axis for top plot if using datetime
    if use_datetime and 'datetime' in episode_df.columns:
        from matplotlib.dates import DateFormatter
        # Format the datetime axis only for the top plot
        formatter = DateFormatter('%H:%M:%S')
        ax1.xaxis.set_major_formatter(formatter)
        # Rotate x-axis labels for the top plot
        for label in ax1.get_xticklabels():
            label.set_rotation(45)
    
    plt.tight_layout()
    
    if save_path:
        fig.savefig(save_path, dpi=300, bbox_inches='tight')

    if show:
        plt.show()
    return fig

In [10]:
# Example: Plot a specific episode with its mutual information and extra cost
example_episode_idx = 300  # Change this to plot different episodes

if example_episode_idx < len(episode_info_dfs):
    episode_df = episode_info_dfs[example_episode_idx]
    episode_idx = episode_df['episode_index'].iloc[0] if 'episode_index' in episode_df.columns else example_episode_idx
    mi_value = mutual_info_values[example_episode_idx]
    extra_cost = extra_cost_values[example_episode_idx]
    
    print_log(f"Plotting episode {episode_idx} with MI = {mi_value:.6f} and Extra Cost = ${extra_cost:.4f}")
    
    # Plot with datetime
    fig = plot_episode_with_psd(episode_df, episode_idx, mi_value, extra_cost, use_datetime=True, show=True)
    
    # Optionally save the plot
    # save_path = expt_folder / "graphs" / "mi_analysis" / f"episode_{episode_idx}_with_mi.png"
    # if not save_path.parent.exists():
    #     save_path.parent.mkdir(parents=True)
    # fig.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.close('all')
else:
    print_log(f"Episode index {example_episode_idx} is out of range. Available episodes: 0 to {len(episode_info_dfs)-1}")

[2025-07-24 17:27:02:276] Plotting episode 42 with MI = 1.213129 and Extra Cost = $0.9132


/tmp/ipykernel_727201/1765947300.py:116: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [11]:
# Create a summary plot of mutual information across all episodes
fig, ax = plt.subplots(1, 1, figsize=(16, 8), dpi=150)

# Plot mutual information over episodes with light blue color
episode_indices = range(len(mutual_info_values))
sns.lineplot(x=episode_indices, y=mutual_info_values, ax=ax, label='Mutual Information', color='blue', alpha=0.3, linewidth=2)

# Add rolling average (similar to expt_results.ipynb style)
window_size = 100  # Rolling window size
mi_series = pd.Series(mutual_info_values)
mi_rolling = mi_series.rolling(window=window_size, min_periods=1).mean()

sns.lineplot(x=episode_indices, y=mi_rolling, ax=ax, 
             label=f'MI Rolling Average (window={window_size})', 
             color='blue', linewidth=3, alpha=1.0)

ax.set_title('Mutual Information Between Masked Grid Load and User Load Across Episodes')
ax.set_xlabel('Episode Index')
ax.set_ylabel('Mutual Information')
ax.legend()
ax.grid(True, alpha=0.3)

# Add statistics as text with better alignment and transparent background
stats_text = f"""Statistics:
Mean MI : {np.nanmean(mutual_info_values):.6f}
Std MI    : {np.nanstd(mutual_info_values):.6f}
Min MI    : {np.nanmin(mutual_info_values):.6f}
Max MI   : {np.nanmax(mutual_info_values):.6f}"""

ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='none', edgecolor='black', alpha=0.8))

plt.tight_layout()

# Save the summary plot
mi_summary_path = expt_folder / "graphs" / "mi_summary_across_episodes.png"
if not mi_summary_path.parent.exists():
    mi_summary_path.parent.mkdir(parents=True)
fig.savefig(mi_summary_path, dpi=300, bbox_inches='tight')
print_log(f"Summary plot saved to: {mi_summary_path}")

plt.show()


[2025-07-24 17:27:02:759] Summary plot saved to: experiments/20250723_183309/logs_train/graphs/mi_summary_across_episodes.png


/tmp/ipykernel_727201/3181371295.py:42: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
# Function to plot multiple episodes with their MI values and PSD
def plot_multiple_episodes_with_psd(episode_info_dfs, mutual_info_values, extra_cost_values, episode_indices=None, 
                                   save_folder=None, use_datetime=True, show_plots=False):
    """
    Plot multiple episodes with their mutual information values and PSD analysis
    
    Args:
        episode_info_dfs: List of episode DataFrames
        mutual_info_values: List of mutual information values
        extra_cost_values: List of extra cost values
        episode_indices: List of episode indices to plot (if None, plots first 10)
        save_folder: Folder to save plots (if None, doesn't save)
        use_datetime: Whether to use datetime for x-axis
        show_plots: Whether to display plots
    """
    if episode_indices is None:
        episode_indices = list(range(min(10, len(episode_info_dfs))))
    
    print_log(f"Plotting {len(episode_indices)} episodes with MI and extra cost values...")
    
    for ittr, ep_idx in enumerate(episode_indices):
        if ep_idx >= len(episode_info_dfs):
            print_log(f"Episode index {ep_idx} out of range, skipping...")
            continue
            
        episode_df = episode_info_dfs[ep_idx]
        episode_idx = episode_df['episode_index'].iloc[0] if 'episode_index' in episode_df.columns else ep_idx
        mi_value = mutual_info_values[ep_idx]
        extra_cost = extra_cost_values[ep_idx]
        
        # Create save path if folder provided
        save_path = None
        if save_folder:
            save_path = save_folder / f"episode_{ittr + 1:04d}_with_MI_and_cost.png"
            if not save_path.parent.exists():
                save_path.parent.mkdir(parents=True)
        
        # Plot the episode
        fig = plot_episode_with_psd(episode_df, episode_idx, mi_value, extra_cost,
                                   save_path=save_path, use_datetime=use_datetime, show=show_plots)
        
        # Close figure to save memory
        if not show_plots:
            plt.close(fig)

        if (ittr + 1) % 5 == 0:
            print_log(f"Completed {ittr + 1}/{len(episode_indices)} episodes")

# Example usage: Plot first 5 episodes and some selected episodes
selected_episodes = [0, 1, 2, 3, 4]  # First 5 episodes

# You can also select episodes with specific MI characteristics
# For example, episodes with highest MI:
# mi_sorted_indices = np.argsort(mutual_info_values)[::-1]  # Sort descending
# selected_episodes = mi_sorted_indices[:5].tolist()  # Top 5 highest MI

# all episodes
selected_episodes = list(range(len(episode_info_dfs)))  # All episodes

# Plot the selected episodes
mi_plots_folder = expt_folder / "graphs" / "MI_analysis"
plot_multiple_episodes_with_psd(episode_info_dfs, mutual_info_values, extra_cost_values,
                               episode_indices=selected_episodes,
                               save_folder=mi_plots_folder,
                               use_datetime=True, 
                               show_plots=False)  # Set to True if you want to see plots

print_log(f"Episode plots with MI saved to: {mi_plots_folder}")

[2025-07-24 17:27:02:775] Plotting 605 episodes with MI and extra cost values...
[2025-07-24 17:27:05:649] Completed 5/605 episodes
[2025-07-24 17:27:09:259] Completed 10/605 episodes
[2025-07-24 17:27:12:197] Completed 15/605 episodes
[2025-07-24 17:27:15:085] Completed 20/605 episodes
[2025-07-24 17:27:17:943] Completed 25/605 episodes
[2025-07-24 17:27:20:685] Completed 30/605 episodes
[2025-07-24 17:27:23:546] Completed 35/605 episodes
[2025-07-24 17:27:26:273] Completed 40/605 episodes
[2025-07-24 17:27:29:060] Completed 45/605 episodes
[2025-07-24 17:27:31:930] Completed 50/605 episodes
[2025-07-24 17:27:34:641] Completed 55/605 episodes
[2025-07-24 17:27:37:517] Completed 60/605 episodes
[2025-07-24 17:27:40:885] Completed 65/605 episodes
[2025-07-24 17:27:43:677] Completed 70/605 episodes
[2025-07-24 17:27:46:580] Completed 75/605 episodes
[2025-07-24 17:27:49:320] Completed 80/605 episodes
[2025-07-24 17:27:52:195] Completed 85/605 episodes
[2025-07-24 17:27:54:901] Completed 

## Mutual Information Analysis Results

The mutual information (MI) metric measures the amount of information that the masked grid load reveals about the user load. Lower MI values indicate better privacy protection.

Key insights from the analysis:
- **MI = 0**: Perfect privacy - masked grid load reveals no information about user load
- **Higher MI values**: More information leakage - the masking is less effective
- **MI trends over episodes**: Shows how the RL agent learns to balance privacy vs. cost

The plots above show:
1. **Load profiles**: Comparison between actual user load and masked grid load
2. **MI visualization**: The mutual information level for each episode, displayed as a constant line since MI is calculated per episode

This analysis helps evaluate the privacy-preserving effectiveness of the adversarial smart meter control system.